Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as any collaborators you worked with:

In [ ]:
COLLABORATORS = ""

---

In [ ]:
%matplotlib inline
%precision 16
import numpy
import matplotlib.pyplot as plt
import pandas as pd

# HW 8:  Linear Algebra Fun with the QR and the SVD

## Question 1: Fun with the QR

**(a)** [4] - Why we use the $QR$ factorization for least-squares.

Consider the two equivalent problems to solve for the least-squares solution $\hat{\mathbf{x}}$

The Normal Equations
$$
    A^TA\hat{\mathbf{x}} = A^T\mathbf{b}
$$

and using the $QR$ factorization with $A=QR$
$$
\quad R\hat{\mathbf{x}}=Q^T\mathbf{b}
$$


* Show that the condition number of $A^TA$ is the square of that of $A$ (i.e.$\kappa_2(A^T A)$ = $\kappa_2^2(A)$).  Hint: use the SVD $A=U\Sigma V^T$
* Show that $\kappa_2(R) = \kappa_2(A)$

so that solving using the $QR$ factorization is much more stable with respect to floating point error.

YOUR ANSWER HERE

**(b)** [8] - The Householder reflection matrix, 

$$
    H = I - 2\mathbf{q}\mathbf{q}^T
$$

reflects a vector $x$ across a mirror plane normal to a unit vector $\mathbf{q}$ and is an essential ingredient in many numerical methods.

Demonstrate the following properties of $H$

* $H$ is symmetric
* $H$ is Unitary
* $H$ is not a projection matrix
* if $\mathbf{x}$ is in $\mathbb{R}^n$,  the Matrix-vector product $H\mathbf{x}$ can be computed in $O(n)$ operations
* Repeated application of householder matrices to transform $A\rightarrow R$ do not change the condition number (i.e. $\kappa_2(HA) = \kappa_2(A) = \kappa_2(R)$

YOUR ANSWER HERE

**(c)** [6] - To understand the basic algorithms better,  use the modified Gram-Schmidt algorithm 

$$A\rightarrow Q,\quad   R = Q^T A$$

to construct, by hand, matrices $Q$ and $R$ such that $A=QR$ for 

$$
    A = \begin{bmatrix} 1 & 1 & 1\\ 1 & 0 & 0 \\ 1 & 1 & 0  \\ 1 & 0 & 0 \\ \end{bmatrix} 
$$

You can use any python/numpy routines to check your answers

YOUR ANSWER HERE

**(c)** [4]  Householder Triangulation is a sequence of Unitary transformations $Q_1$, $Q_2$, $Q_3$, that transform $A$ to logically upper triangular form (i.e. is zero for all elements below the diagonal) using Householder reflection matrices.  Construct by hand

* the first Unitary matrix  $Q_1$ that zeros out the first column of A below the diagonal and calculate $A_1 = Q_1A$. (you can use numpy to check your solution).   **Hint**: there are two ways to calculate the normal to the reflection plane 
$\mathbf{v}$.  Your life will be 
much easier if you use the simpler version 

$$
    \mathbf{v} = ||\mathbf{x}||\mathbf{e}_1 - \mathbf{x}
$$

* the second unitary matrix $Q_2$ that preserves your first column but puts zeros below the diagonal in the second column.  You can use numpy to calculate $Q_2Q_1A$ to check your answer.

* **Extra Credit** (2 pts) Work out $Q3$  and $R=Q_3Q_2Q_1A$ and compare to the solution by Modified Gram-schmidt

YOUR ANSWER HERE

In [ ]:
# you can add code to check your answer here if you want

**(d)** [10] - Modify the $QR$ factorization by Householder reflection given in class `householder_QR` to write a function to solve linear least-squares problems, i.e. use repeated Householder reflections to transform

$$
    A\mathbf{x} = \mathbf{b}
$$

to

$$ R\mathbf{x} = \mathbf{c} $$

where $\mathbf{c} = Q^T\mathbf{b}$.  Then solve the last equation for $\mathbf{x}$ using `numpy.linalg.solve` (which implements a $LU$ decomposition for Gaussian elimination).


In [ ]:
# Implementation of Householder QR, for least-squares
def mylstq(A, b):
    """
    Solves the leastsquares problem Ax = b, by Householder reduction to Rx = c,
    then solve Rx = c, using numpy.linalg.solve()
    
    fill in the rest of the doc-string properly
    
    usage:  x = mylstq(A, b)
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return x

In [ ]:
# Test this on the previous problem
A = numpy.array([ [ 1., 1., 1.],
                  [1., 0., 0.],
                  [1., 1., 0.],
                  [1., 0., 0.]])
b = numpy.array([ 3., -2., 1., 0.])

x = mylstq(A,b)
x_np = numpy.linalg.lstsq(A,b,rcond = None)[0]
print('my solution = {}'.format(x))
print('numpy.lstq  = {}'.format(x_np))
numpy.testing.assert_allclose(x, x_np)
print('Success!')

**(e)** [4] - **Extra Credit** Use your routine to find the best fit of the function 

$$
    y = c_1 x + c_2e^{x}
$$ 
through the $(x, y)$ data points $(1,2)$, $(2,3)$, $(3,5)$, $(4,10)$, $(5,15)$ and make a plot comparing the best fit function to the data over the interval $x \in [0, 6]$.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Question 2 - Hidden Figures

A little fun with the SVD.  Here we are going to use it extract low-dimensional information embedded in high dimensions.

The following cells will read in a matrix of 46765 samples of 5-dimensional data and make a series of scatter plots comparing the data along each dimension pairwise.

In [ ]:
data = pd.read_csv('data.csv.gz').values
print('shape = {}'.format(data.shape))

In [ ]:
# plot scatter diagrams

fig = plt.figure(figsize=(20,6))
for i in range(4):
    axes = fig.add_subplot(1,4,i+1)
    axes.scatter(data[:,i],data[:,i+1],s=1)
    axes.set_xlabel('x_{}'.format(i), fontsize=16)
    axes.set_ylabel('x_{}'.format(i+1),fontsize=16)
    axes.grid()
plt.show()

**(a)** [6] Now demean the data and use the SVD to determine the dimension of the subspace of $\mathbb{R}^5$ that contains the data. Making a plot of the singular values will help. (hint: you will also want to use the `full_matrices=False` argument to the SVD to get the skinny SVD and save a lot of computation and memory)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

**(b)** [4]  Principal Components. Make a 2-D scatter plot that projects the data onto the plane spanned by the first two principal components (singular vectors of $V$).  and comment.  (**Extra Credit** do this in 3-D)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()